In [ ]:
import sys,os,tempfile

from larlite import larlite                                                                                                   
from larlite import larutil                                                                                                    
__sce=larutil.SpaceChargeMicroBooNE() 

from larcv import larcv
import ROOT

import matplotlib
matplotlib.rcParams['font.size']=20
matplotlib.rcParams['font.family']='serif'
from matplotlib import pyplot as plt
import numpy as np

from ROOT import geo2d
pygeo = geo2d.PyDraw()

%matplotlib inline


# Configuration
#### *) set input file (INFILE)
#### *) set PGraph producer  -- PGRAPH_PRODUCER
#### *) set Pixel2D producer for contour  --- PIXCTOR_PRODUCER
#### *) set the MC ROI producer  --- MCROI_PRODUCER

In [ ]:
INFILE="/Users/vgenty/Desktop/intrinsic_nue/out_pid_merge_mask3/larcv_pid_0000_0099.root"
ROI_PRODUCER    ="croimerge"
PGRAPH_PRODUCER ="test"
PIXCTOR_PRODUCER="test_ctor"
MCROI_PRODUCER  ="tpc"

# ProcessDriver configuration

In [ ]:
cfg="""
ProcessDriver:{
  Verbosity: 0
  EnableFilter: false
  RandomAccess: false
  ProcessType: ["ChannelMax","ChannelMax","ChannelMax","CombineImages","SegmentMask","SegmentMask","MaskImage","MaskImage","LArbysImageExtract"]
  ProcessName: ["ChannelMaxPlane0","ChannelMaxPlane1","ChannelMaxPlane2","CombineImages","ShowerSegment","TrackSegment","ShowerImage","TrackImage","LArbysImageExtract"]
  AnaFile: "/tmp/_.root"
  IOManager: {
    Verbosity: 0
    Name:         "IOManager"
    IOMode:        2
    OutFileName:   "/tmp/__.root"
    StoreOnlyType: [0]
    StoreOnlyName: ["_"]
  }
  ProcessList: {
  ChannelMaxPlane0: {
      Verbosity: 2
      InProducer: "uburn_plane0"
      NPlanes: 2
      OutputProducer: "uburn_max_plane0"
      ChannelWeights: [1,1]
      RelevantChannels: [0,1]
      ChannelMask: [1,2]
   }
   ChannelMaxPlane1: {
      Verbosity: 2
      InProducer: "uburn_plane1"
      NPlanes: 2
      OutputProducer: "uburn_max_plane1"
      ChannelWeights: [1,1]
      RelevantChannels: [0,1]
      ChannelMask: [1,2]
   }
  ChannelMaxPlane2: {
      Verbosity: 2
      InProducer: "uburn_plane2"
      NPlanes: 2
      OutputProducer: "uburn_max_plane2"
      ChannelWeights: [1,1]
      RelevantChannels: [0,1]
      ChannelMask: [1,2]
   }                                                                                                                     
   CombineImages: {
      Verbosity: 2
      ImageProducers: ["uburn_max_plane0","uburn_max_plane1","uburn_max_plane2"]
      NPlanes: 1
      OutputProducer: "uburn"
    }
    ShowerSegment: {
      Verbosity: 2
      ROIProducer: ""
      ImageProducer:  "uburn"
      OutputProducer: "shower_segment"
      ValidROI: ["Cosmic"]  
      MaskValue: 0
      PlaneID: [0,1,2]
    }
  TrackSegment: {
      Verbosity: 2
      ROIProducer: ""
      ImageProducer:  "uburn"
      OutputProducer: "track_segment"
      ValidROI: ["BNB"]
      MaskValue: 0
      PlaneID: [0,1,2]
    }
    ShowerImage: {
      Verbosity: 2
      MinPIThreshold: 1
      MaskValue: 0
      OutputProducer: "shower_tpc"
      ReferenceProducer: "shower_segment"
      TargetProducer: "tpc"
    }
    TrackImage: {
      Verbosity: 2
      MinPIThreshold: 1
      MaskValue: 0
      OutputProducer: "track_tpc"
      ReferenceProducer: "track_segment"
      TargetProducer: "tpc"
    }
    LArbysImageExtract: {
      ADCImageProducer: "tpc"
      TrackImageProducer: "track_tpc"
      ShowerImageProducer: "shower_tpc"
      ThruMuProducer: "thrumupixels"
      StopMuProducer: "stopmupixels"
      PreProcessor: {
         Verbosity: 2                                                                                                          
         PiThreshold: 1                                                                                                        
         MinContourSize: 4                                                                                                     
         MinTrackSize: 3                                                                                                       
         AllowedNeighborSeparation: 10                                                                                         
         BlurSize: 4                                                                                                           
         EdgePCABoxSize: 5                                                                                                     
         MinPCAOverallAngle: 10                                                                                                
      }
      LArbysImageMaker: {
        Q2Gray: 2
        QMax: 500
        QMin: 0
      }
    }
  }
}
"""
fout=tempfile.NamedTemporaryFile(mode='w+',delete=False)
fout.write(cfg)
fout.close()

In [ ]:
proc=larcv.ProcessDriver('ProcessDriver')
proc.configure(fout.name)
proc.override_input_file(ROOT.std.vector("string")(1,ROOT.std.string(INFILE)))
proc.initialize()  
iom      = proc.io()
ext_id   = proc.process_id("LArbysImageExtract")                                                                                                                          
ext_proc = proc.process_ptr(ext_id) 

In [ ]:
def Project3D(meta,x,y,z,t,plane,sce=False):                                                                                             
    vec=[0,0,0]
    offset=0.0
    if sce:
        vec=__sce.GetPosOffsets(x,y,z)
        offset=0.7
    x=x - vec[0] + offset
    y=y + vec[1]
    z=z + vec[2]
    geohelp = larutil.GeometryHelper.GetME()
    larpro  = larutil.LArProperties.GetME()
    vtx_2d = geohelp.Point_3Dto2D(x, y, z, plane)
    x_compression  = meta.width()  / meta.cols()
    y_compression  = meta.height() / meta.rows()
    xpixel = (vtx_2d.w/geohelp.WireToCm() - meta.tl().x) / x_compression
    ypixel = (((x/larpro.DriftVelocity() + t/1000.)*2+3200)-meta.br().y)/y_compression
    return (xpixel,ypixel)

def segment_image(meta_v,preprocess,cosmicid):
    img_v=[None,None,None]
    
    for plane in xrange(3):

        trackimg2d  = ext_proc.trk_img(plane)                                                                                                           
        showerimg2d = ext_proc.shr_img(plane)
        trackimg2d = trackimg2d.crop(meta_v[plane])
        showerimg2d = showerimg2d.crop(meta_v[plane])
    
        trackimg = ext_proc.maker().ExtractMat(trackimg2d)
        showerimg = ext_proc.maker().ExtractMat(showerimg2d)

        if preprocess:
            adcimg2d=ext_proc.adc_img(plane) 
            adcimg2d=adcimg2d.crop(meta_v[plane])
            adcimg  =ext_proc.maker().ExtractMat(adcimg2d)
            ext_proc.pproc().PreProcess(adcimg,trackimg,showerimg)
        
        if cosmicid:
            adcimg2d=ext_proc.adc_img(plane)
            stopmuimg2d= ext_proc.maker().ConstructCosmicImage(ext_proc.ev_stopmu_pix(),
                                                               adcimg2d,plane,100.0)
            thrumuimg2d= ext_proc.maker().ConstructCosmicImage(ext_proc.ev_thrumu_pix(),
                                                               adcimg2d,plane,100.0)
            stopmuimg2d = stopmuimg2d.crop(meta_v[plane])
            thrumuimg2d = thrumuimg2d.crop(meta_v[plane])
            
            stopmuimg = ext_proc.maker().ExtractMat(stopmuimg2d)
            thrumuimg = ext_proc.maker().ExtractMat(thrumuimg2d)
            
            smuimg=pygeo.image(stopmuimg)
            tmuimg=pygeo.image(thrumuimg)
            
            smuimg=np.where(smuimg>0.0,0.0,1.0)
            tmuimg=np.where(tmuimg>0.0,0.0,1.0)
        
        timg=pygeo.image(trackimg)                                                                                                                                   
        simg=pygeo.image(showerimg)                                                                                                                                  

        timg=np.where(timg>10,200,1.0)                                                                                                                               
        simg=np.where(simg>10,100,1.0)                                                                                                                            
        img_v[plane]=simg.copy()+timg.copy()
        if cosmicid:
            img_v[plane]*=smuimg
            img_v[plane]*=tmuimg
        
        img_v[plane]=img_v[plane].T[::-1,:]
    return img_v

# Entry selection

In [ ]:
event=13
if iom.current_entry()==event:
    proc.process_entry(event+1)
proc.process_entry(event)
print "Found...",iom.get_data(larcv.kProductPGraph,PGRAPH_PRODUCER).PGraphArray().size(),"vertices"

# <==== All ROIs in Event ====>
## (cyan = Shower & orange = Track)
### PreProcessor toggle available

In [ ]:
mycmap = matplotlib.cm.get_cmap('jet')
mycmap.set_under('w')

preprocess=True
cosmicid  =True

evroi=iom.get_data(larcv.kProductROI,ROI_PRODUCER).ROIArray()
for roi_id,roi in enumerate(evroi):
    print "<=================================================================>"
    print roi_id,")",roi
    
    img_v=segment_image(roi.BB(),preprocess,cosmicid)       
    
    for plane in xrange(3):
        fig,ax=plt.subplots(figsize=(10,12))
        ax.imshow(img_v[plane],cmap=mycmap,vmin=1,vmax=255,interpolation='none')
        ax.set_title("Plane {}".format(plane),fontsize=30)
        plt.show()
    print "<=================================================================>"

In [ ]:
colors=['magenta','white','yellow']

evroi    = iom.get_data(larcv.kProductROI,MCROI_PRODUCER)
evpgraph = iom.get_data(larcv.kProductPGraph,PGRAPH_PRODUCER)
evctor   = iom.get_data(larcv.kProductPixel2D,PIXCTOR_PRODUCER)

for vertex_id, pgraph in enumerate(evpgraph.PGraphArray()):
    print "<=================================================================>"
    print "PGraph ID:",vertex_id,"@",pgraph
    pararray=pgraph.ParticleArray()
    
    img_v = segment_image(pararray.back().BB(),True,False)
    ctor_v=[]
    for parid,id_ in enumerate(np.array(pgraph.ClusterIndexArray())):
        pix2d_vv=[]
        for plane in xrange(3):
            pix2d_v=evctor.Pixel2DClusterArray(plane).at(id_)
            pix2d_v=[[pix2d_v[i_].X(),pix2d_v[i_].Y()] for i_ in xrange(pix2d_v.size())]
            if len(pix2d_v)!=0:
                pix2d_v.append(pix2d_v[0])
            pix2d_vv.append(np.array(pix2d_v))
        ctor_v.append(pix2d_vv)
        
    for plane in xrange(3):
        SS="Plane {}".format(plane)
        fig,ax=plt.subplots(figsize=(20,22))
        ax.imshow(img_v[plane],cmap='jet',vmin=0,vmax=255,interpolation='none')
        for ix,ctor in enumerate(ctor_v):
            ctor=ctor[plane]
            if ctor.size==0: continue
            ax.plot(ctor[:,0],ctor[:,1],'-o',lw=3,color=colors[ix],alpha=1.0)
        par=pararray.back()
        reco_x_=par.X()
        reco_y_=par.Y()
        reco_z_=par.Z()
        reco_vtx_x,reco_vtx_y = Project3D(par.BB(plane),reco_x_,reco_y_,reco_z_,0,
                                          plane,False) 
        reco_vtx_y=par.BB(plane).rows()-reco_vtx_y
        ax.plot(reco_vtx_x,reco_vtx_y,'*',markersize=30,color='yellow')
        
    
        plane_nu = evroi.ROIArray().front()
        true_x_=plane_nu.X()
        true_y_=plane_nu.Y()
        true_z_=plane_nu.Z()
        true_t_=plane_nu.T()
        true_vtx_x,true_vtx_y = Project3D(par.BB(plane),
                                          true_x_,true_y_,true_z_,true_t_,
                                          plane,True) 

        if (true_vtx_x>par.BB(plane).cols() or true_vtx_y>par.BB(plane).rows()):
            SS+=" -- VERTEX OUTSIDE ROI"
        else:
            true_vtx_y=par.BB(plane).rows()-true_vtx_y
            SS+=" -- VERTEX @ (%.0f,%.0f)"%(true_vtx_x,true_vtx_y)
            ax.plot(true_vtx_x,true_vtx_y,'*',markersize=30,color='pink')
        ax.set_title(SS,fontsize=30)
        ax.grid()
        plt.show()
    print "<=================================================================>"